In [4]:
!pip install datasets

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install torch soundfile ruamel.yaml omegaconf nemo-toolkit datasets gdown gradio
!pip install lhotse jiwer pytorch-lightning nemo-asr hydra-core ruamel.yaml webdataset pyannote.audio

  Using cached lhotse-1.29.0-py3-none-any.whl.metadata (17 kB)
  Using cached jiwer-3.0.5-py3-none-any.whl.metadata (2.7 kB)
  Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl.metadata (21 kB)
  Using cached nemo_asr-0.9.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached webdataset-0.2.100-py3-none-any.whl.metadata (12 kB)
  Using cached pyannote.audio-3.3.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached cytoolz-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.6 kB)
  Using cached intervaltree-3.1.0.tar.gz (32 kB)
  Preparing metadata (setup.py) ... done
  Using cached lilcom-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached rapidfuzz-3.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached torchmetrics-1.6.1-py3-none-any.whl.metadata (21 kB)
  Using cached lightning_utilities-0.11.9-py3

In [2]:
!git clone https://github.com/AbdelrhmanElnenaey/ASR_for_egyptian_dialect
%cd ASR_for_egyptian_dialect


Cloning into 'ASR_for_egyptian_dialect'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 588 (delta 108), reused 35 (delta 17), pack-reused 406 (from 1)
Receiving objects: 100% (588/588), 31.70 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (317/317), done.
/content/ASR_for_egyptian_dialect/ASR_for_egyptian_dialect


In [3]:
import os
ckpt_path = "/content/drive/MyDrive/asr_model.ckpt"
if not os.path.exists(ckpt_path):
    !gdown --fuzzy https://drive.google.com/uc?id=1faLSvzXVcZd_lvBXxxdWYyBGyGnC2ijL -O asr_model.ckpt

config_path = "/content/ASR_for_egyptian_dialect/configs/FC-transducer-inference.yaml"
if not os.path.exists(config_path):
    !wget -O FC-transducer-inference.yaml https://github.com/AbdelrhmanElnenaey/ASR_for_egyptian_dialect/blob/main/configs/FC-transducer-inference.yaml


Downloading...
From (original): https://drive.google.com/uc?id=1faLSvzXVcZd_lvBXxxdWYyBGyGnC2ijL
From (redirected): https://drive.google.com/uc?id=1faLSvzXVcZd_lvBXxxdWYyBGyGnC2ijL&confirm=t&uuid=64675bb4-51db-4617-b27b-c65bdb56dce8
To: /content/ASR_for_egyptian_dialect/asr_model.ckpt
100% 365M/365M [00:09<00:00, 39.2MB/s]


In [6]:
!pip install hydra-core pytorch-lightning lightning lhotse jiwer pytorch-lightning nemo-asr hydra-core ruamel.yaml webdataset pyannote.audio

In [3]:
import torch
import librosa
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML
from omegaconf import OmegaConf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ASR model
def load_asr_model(ckpt_path):
    config_path = '/content/ASR_for_egyptian_dialect/configs/FC-transducer-inference.yaml'
    yaml = YAML(typ='safe')
    with open(config_path) as f:
        params = yaml.load(f)
    params['model'].pop('test_ds', None)
    conf = OmegaConf.create(params)

    model = nemo_asr.models.EncDecRNNTBPEModel(cfg=conf['model']).to(device)
    model.load_state_dict(torch.load(ckpt_path)['state_dict'])
    model.eval()
    return model


In [8]:
# Load the ASR model
asr_model = load_asr_model("//content/ASR_for_egyptian_dialect/asr_model.ckpt")

# Load the audio and perform inference
audio_path = '/content/ASR_for_egyptian_dialect/assets/00b906eba080054a.wav'  # Replace with your own file path
audio, sr = librosa.load(audio_path, sr=16000)

with torch.no_grad():
    transcript = asr_model.transcribe([audio])

print(f"Transcript: {transcript[0]}")


[NeMo I 2025-01-07 00:05:06 mixins:176] Tokenizer SentencePieceTokenizer initialized with 256 tokens


[NeMo W 2025-01-07 00:05:06 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'max_duration': 16.7, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'fully_randomized', 'bucketing_batch_size': None}
[NeMo W 2025-01-07 00:05:06 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False, 'num_workers': 8, 'pin_memory': True}


[NeMo I 2025-01-07 00:05:06 features:305] PADDING: 0
[NeMo I 2025-01-07 00:05:07 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-01-07 00:05:07 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-01-07 00:05:07 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s][NeMo W 2025-01-07 00:05:23 deprecated:65] Function ``_transcribe_output_processing`` is deprecated. The return type of args will be updated in the upcoming release to ensure a consistent             output format across all decoder types, such that a "Hypothesis" object is always returned.

Transcribing: 100%|██████████| 1/1 [00:02<00:00,  2.48s/it]

Transcript: ['إيه أحلى مكان لشراء الهدايه التذكاريه']


In [4]:
import torch
import librosa
from ruamel.yaml import YAML
from omegaconf import OmegaConf
import nemo.collections.asr as nemo_asr
import gradio as gr
import gdown
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to download the ASR model if it doesn't exist locally
def download_asr_model(ckpt_url, output_path):
    if not os.path.exists(output_path):
        print(f"Downloading model from {ckpt_url}...")
        gdown.download(ckpt_url, output_path, quiet=False)
    else:
        print(f"Model already exists at {output_path}")

# Function to load the ASR model
def load_asr_model(ckpt_path, config_path):
    yaml = YAML(typ="safe")
    with open(config_path) as f:
        params = yaml.load(f)
    params["model"].pop("test_ds", None)
    conf = OmegaConf.create(params)
    model = nemo_asr.models.EncDecRNNTBPEModel(cfg=conf["model"]).to(device)
    model.load_state_dict(torch.load(ckpt_path)["state_dict"])
    model.eval()
    return model

# Function to transcribe audio
def transcribe(audio_path):
    try:
        audio, sr = librosa.load(audio_path, sr=16000)  # Load audio at 16kHz
        with torch.no_grad():
            transcript = asr_model.transcribe([audio])
        return transcript[0]
    except Exception as e:
        return f"Error during transcription: {e}"

# Paths to model files
ckpt_url = "https://drive.google.com/uc?id=1faLSvzXVcZd_lvBXxxdWYyBGyGnC2ijL"  # Use the link for downloading
ckpt_path = "/content/ASR_for_egyptian_dialect/asr_model.ckpt"
config_path = "/content/ASR_for_egyptian_dialect/configs/FC-transducer-inference.yaml"

# Download the ASR model if not present
download_asr_model(ckpt_url, ckpt_path)

# Load the ASR model
print("Loading ASR model...")
asr_model = load_asr_model(ckpt_path, config_path)
print("ASR model loaded successfully.")

# Gradio inference functions
def infer_from_upload(uploaded_audio):
    return transcribe(uploaded_audio)

def infer_from_mic(mic_audio):
    return transcribe(mic_audio)

# Gradio interface
interface = gr.Interface(
    fn=infer_from_upload,
    inputs=gr.Audio(type="filepath", label="Upload Audio"),
    outputs="text",
    title="Speech-to-Text System (Upload)",
    description="Upload an audio file to transcribe speech to text.",
)

mic_interface = gr.Interface(
    fn=infer_from_mic,
    inputs=gr.Audio(type="filepath", label="Record Audio"),
    outputs="text",
    title="Speech-to-Text System (Microphone)",
    description="Record audio using your microphone to transcribe speech to text.",
)

# Combine both interfaces
combined_interface = gr.TabbedInterface(
    interface_list=[interface, mic_interface],
    tab_names=["Upload Audio", "Record Audio"],
)

# Launch the Gradio app
if __name__ == "__main__":
    combined_interface.launch()


Model already exists at /content/ASR_for_egyptian_dialect/asr_model.ckpt
Loading ASR model...
[NeMo I 2025-01-07 00:07:38 mixins:176] Tokenizer SentencePieceTokenizer initialized with 256 tokens


[NeMo W 2025-01-07 00:07:39 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'max_duration': 16.7, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'fully_randomized', 'bucketing_batch_size': None}
[NeMo W 2025-01-07 00:07:39 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False, 'num_workers': 8, 'pin_memory': True}


[NeMo I 2025-01-07 00:07:39 features:305] PADDING: 0
[NeMo I 2025-01-07 00:07:40 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-01-07 00:07:40 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-01-07 00:07:40 rnnt_models:226] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
ASR model loaded successfully.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87008d832753190b44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working director